In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from pydrake.all import (
    ModelVisualizer,
    StartMeshcat,
    Simulator,
    DiagramBuilder,
)

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
import matplotlib.pyplot as plt
import numpy as np

import sponana.utils
from sponana.perception import (
    BananaSpotter,
    add_camera_pose_extractor,
    add_body_pose_extractor,
)
from sponana.controller import SpotController, solve_ik
from sponana.debug_logger import DebugLogger
from sponana.planner import Navigator

In [38]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


In [39]:
inspect_spot_model = False

if inspect_spot_model:
    # Inspecting the Spot Model
    visualizer = ModelVisualizer(meshcat=meshcat)
    sponana.utils.configure_parser(visualizer.parser())
    visualizer.AddModels(
        url="package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf"
    )
    visualizer.Run(loop_once=not running_as_notebook)
    meshcat.DeleteAddedControls()

In [40]:
def create_scene(scenario_data: str, debug: bool = True):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    scenario = load_scenario(data=scenario_data)
    station = builder.AddSystem(
        MakeHardwareStation(
            scenario, meshcat, parser_preload_callback=sponana.utils.configure_parser
        )
    )

    spot_plant = station.GetSubsystemByName(
        "spot.controller"
    ).get_multibody_plant_for_control()
    spot_controller = builder.AddSystem(
        SpotController(spot_plant, meshcat=meshcat, use_teleop=False)
    )
    builder.Connect(
        spot_controller.get_output_port(),
        station.GetInputPort("spot.desired_state"),
    )

    # planner
    planner = builder.AddNamedSystem("navigator", Navigator(meshcat=meshcat))
    builder.Connect(
        station.GetOutputPort("spot.state_estimated"),
        planner.get_spot_state_input_port(),
    )
    builder.Connect(
        planner.get_output_port(),
        spot_controller.GetInputPort("desired_base_position"),
    )

    # Get camera poses
    spot_camera_config = scenario.cameras["spot_camera"]
    camera_pose_extractor = add_camera_pose_extractor(
        spot_camera_config, station, builder
    )

    # Camera to Perception
    spot_camera = station.GetSubsystemByName("rgbd_sensor_spot_camera")
    banana_spotter = builder.AddSystem(BananaSpotter(spot_camera))
    banana_spotter.set_name("banana_spotter")
    builder.Connect(
        station.GetOutputPort("spot_camera.rgb_image"),
        banana_spotter.get_color_image_input_port(),
    )
    builder.Connect(
        station.GetOutputPort("spot_camera.depth_image"),
        banana_spotter.get_depth_image_input_port(),
    )

    # Banana pose (for debugging)
    banana_pose_extractor = add_body_pose_extractor(
        "banana", "banana", station, builder
    )
    builder.Connect(
        banana_pose_extractor.get_output_port(),
        spot_controller.GetInputPort("desired_gripper_pose"),
    )

    if debug:
        # Connect debugger
        debugger = builder.AddNamedSystem(
            "debug_logger", DebugLogger(spot_camera, meshcat)
        )
        builder.Connect(
            station.GetOutputPort("spot_camera.rgb_image"),
            debugger.get_color_image_input_port(),
        )
        builder.Connect(
            station.GetOutputPort("spot_camera.depth_image"),
            debugger.get_depth_image_input_port(),
        )
        builder.Connect(
            camera_pose_extractor.get_output_port(),
            debugger.get_camera_pose_input_port(),
        )
        builder.Connect(
            station.GetOutputPort("spot.state_estimated"),
            debugger.get_spot_state_input_port(),
        )
        builder.Connect(
            banana_pose_extractor.get_output_port(),
            debugger.get_banana_pose_input_port(),
        )

    diagram = builder.Build()
    diagram.set_name("everything")  # the outmost diagram
    return diagram

In [41]:
scenario_data = """
cameras:
    spot_camera:
        name: spot_camera
        depth: True
        X_PB:
            translation: [0, 0.05, 0]
            base_frame: spot_camera::base
            rotation: !Rpy { deg: [-90, 0, 0] }

directives:
- add_model:
    name: spot
    file: package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf
    default_joint_positions:
        arm_sh1: [-3.1]
        arm_el0: [3.1]
    
- add_model:
    name: banana
    file: package://sponana/banana/banana.sdf
    default_free_body_pose:
        banana:
            # move it in front of Spot 
            translation: [1.45, 0, 0.05]

- add_model:
    name: floor
    file: package://sponana/platform.sdf

- add_weld:
    parent: world
    child: floor::platform_center

- add_model:
    name: spot_camera
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: spot::body
    child: spot_camera::base
    X_PC:
        translation: [0.4, 0, 0]
        rotation: !Rpy { deg: [-30, 0, -90] }

model_drivers:
    spot: !InverseDynamicsDriver {}
"""

In [42]:
diagram = create_scene(scenario_data)
simulator = Simulator(diagram)
sponana.utils.run_simulation(simulator, meshcat, finish_time=-1)

Press Space to log system info
converted q_goal [0.1243116 0.4359377 0.2475   ] converted q_goal [ 0.20894849 -0.47792893  0.2475    ] converted q_goal [-0.4705993  -0.11675488  0.2475    ]
Q_start in Q [[1.00000000e+000 1.50392176e-012 3.15001955e+000]
 [5.25303356e+170 1.43453220e+180 1.12999427e-042]
 [7.20100785e+252 2.65292601e+180 8.41444044e+276]
 ...
 [1.08907937e+242 6.71193017e+020 2.39098342e+117]
 [1.82430496e+170 2.39000516e+261 1.92266514e+030]
 [5.60943720e+165 1.58665787e+166 5.53456404e+280]]
q_sample = q_goal [0.1243116 0.4359377 0.2475   ]
goal_dist: 0.025857112580475003
path node: 0 [1.00000000e+00 1.50392176e-12 3.15001955e+00]
path node: 1 [-0.53818515  0.85626978  0.2475    ]
path node: 2 [0.01191116 0.63619105 0.2475    ]
path node: 3 [ 0.85605921 -0.54781767  0.2475    ]
path node: 4 [-0.68805237  0.54202764  0.2475    ]
path node: 5 [0.35178302 0.87548461 0.2475    ]
path node: 6 [-0.1080207   0.57978465  0.2475    ]
path node: 7 [ 0.68721576 -0.19984365  0.24

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



Press Escape to stop the simulation


: 

In [ ]:
# test banana spotter
context = simulator.get_mutable_context()
banana_spotter = diagram.GetSubsystemByName("banana_spotter")
context = banana_spotter.GetMyContextFromRoot(context)
has_banana = banana_spotter.GetOutputPort("has_banana").Eval(context)
print("has_banana", has_banana)  # this value is currently being hard-coded

In [ ]:
# another way to get the images directly
context = simulator.get_mutable_context()
station = diagram.GetSubsystemByName("station")
context = station.GetMyContextFromRoot(context)
color_image = station.GetOutputPort("spot_camera.rgb_image").Eval(context)
depth_image = station.GetOutputPort("spot_camera.depth_image").Eval(context)
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(color_image.data)
plt.subplot(1, 2, 2)
plt.imshow(depth_image.data)

In [ ]:
# camera intrinsics
spot_camera = station.GetSubsystemByName("rgbd_sensor_spot_camera")
camera_info = spot_camera.depth_camera_info()
print("intrinsics", camera_info.intrinsic_matrix())
print("focal length", camera_info.focal_x(), camera_info.focal_y())
print("field of view", camera_info.fov_x(), camera_info.fov_y())

In [ ]:
sponana.utils.visualize_diagram(diagram, max_depth=1)